# Rogue Player abilities

**Notes:**  
The purpose of the file is to create export table of the characteristics of the character's classes abilities.

In [452]:
import pandas as pd
import numpy as np
import json
import os
import re
from bs4 import BeautifulSoup
import jsonpath_ng as jp
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 300)
import warnings
import openpyxl
warnings.filterwarnings('ignore')
import flatten_json as fj

### Functions

In [453]:
# Fuction to show missed data in a table
def missing_values_tab(df):
    #Count missed values
    mis_val = df.isnull().sum()
    #Count share of missed values of all data 
    mis_val_percent = round(100 * df.isnull().sum() / len(df),2)
    
    mv_table = pd.concat([mis_val, mis_val_percent], axis=1)
    
    mv_table = mv_table.rename(columns = {0 : 'missed_values', 1 : '%_of_all'})
    
    mv_table['data_type'] = df.dtypes
    mv_table = mv_table[mv_table.iloc[:,1]!=0].sort_values(by='missed_values',ascending=False)
    print ("Dataframe contains " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " strings.\n")
    print("It has  " + str(mv_table.shape[0]) +" columns with missed values")    
    return display(mv_table) 

In [454]:
def column_name_formatter(col):
    """used for delete prefixes 'Data_m_' in columns' names
    """ 
    if 'Data' in col:
        parts = col.split('_')
        new_name = parts[-1]
        return new_name
    else:
        return col

In [455]:
def show_uniques(df):
    for col in df.columns[1:-1]:
        try:
            print(col, ':    ', df[col].unique(),'\n')
        except:
            continue

In [456]:
# Функция для очистки строк от префикса '!bp_'
def clear_rows(row):
        
    if re.match('^\!bp_\w+', str(row)):
        tmp = row.split('_')
        new_value = tmp[1]        
        return new_value
    else:
        return row

In [457]:
def camel_case_split(str):
    """Function useful for splitting camelcase words""" 
    return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)

In [458]:
#Function to extract json into dataframes
def create_df(filelist, datatype = None):
    recipes_df = pd.DataFrame()
    filelist_selected = []
    dfs_list = []
    for file in filelist:
        with open(file, encoding='latin-1') as user_file:
            file_contents = user_file.read()
            data = json.loads(file_contents)
            dic = (fj.flatten(data))
            if datatype == None:
                df = pd.DataFrame.from_dict(dic, orient='index')
                dfs_list.append(df)
            else:
                if data['Data']['$type']==datatype:
                    df = pd.DataFrame.from_dict(dic, orient='index')
                    dfs_list.append(df)
                    filelist_selected.append(file)
    for df in dfs_list:
        recipes_df = recipes_df.join(df, how = 'outer',lsuffix = 'x_',rsuffix = 'y_')
    #assign new column names
    column_numbers = list(range(len(recipes_df.columns)))
    recipes_df.columns = column_numbers
    #fill Nans
    #recipes_df = recipes_df.fillna(0)
    #remove prefix '!bp_'
    for col in recipes_df.columns:
        recipes_df[col] = recipes_df[col].apply(clear_rows)  
    #transpose dataframe
    recipes_df = recipes_df.T
    #-------------------------------------
    cols_to_drop = []
    for col in recipes_df.columns:
        if 'Visual' in col or 'FXSettings' in col: #drop columns with visual parameters
            cols_to_drop.append(col)
        elif 'Data_Comment' in col or 'PrototypeLink' in col:  #drop columns with dev comments
            cols_to_drop.append(col)
        elif 'Meta_ShadowDeleted' in col: #drop trash column
            cols_to_drop.append(col)
        elif 'JsonPath' in col or 'PropertyPath' in col or 'OwnerString' in col or 'assetguid' in col: #drop columns with fileways
            cols_to_drop.append(col)
        elif 'UI' in col or 'Sound' in col or 'Icon' in col: #drop columns with sound, ui properties
            cols_to_drop.append(col)
    recipes_df = recipes_df.drop(cols_to_drop, axis = 1)
    if datatype is not None:
        #add column Filepath
        recipes_df['Filepath'] = filelist_selected
    else:
        recipes_df['Filepath'] = filelist
    return recipes_df

In [459]:
# Function to get rid of characters between curly brackets
""" Sometimes not working correctly
def format_description(row):
    if '{' in row:
        row = re.sub(r'\s*{.*}\s*','',row)
    return row
"""
def format_description(row):      
    openining_braces = '\{'
    closing_braces = '\}'
    text_between = '.*?'
    x = re.sub(f'[{openining_braces}]{text_between}[{closing_braces}]','',row)
    #get rid of html tags
    x = re.sub('<[^>]*>',"",x)
    return x

In [460]:
#Function to find columns with string keys
def match_pattern(row):
        if re.match('^\w{8}-\w{4}-\w{4}-\w{4}-\w{12}$', str(row)):
            return True   
        else:
            return False
   
        
match_pattern('ca24785c24384d142a6c22d7585bb3a6, BlueprintWeaponEnchantment')
"""
#finding columns with string keys
stringkey_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(match_pattern).any()
        if a:            
            stringkey_cols.append(col)
print(stringkey_cols)   
    """

'\n#finding columns with string keys\nstringkey_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(match_pattern).any()\n        if a:            \n            stringkey_cols.append(col)\nprint(stringkey_cols)   \n    '

In [461]:
#Function to find columns with asset ids
def find_assetIds_cols(row):
        if re.match('^[a-zA-Z0-9]{32}$', str(row)):
            return True   
        else:
            return False      

"""
#finding columns with asset ids
assetIds_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(find_assetIds_cols).any()
        if a:            
            assetIds_cols.append(col)
print(assetIds_cols)   
    """

'\n#finding columns with asset ids\nassetIds_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(find_assetIds_cols).any()\n        if a:            \n            assetIds_cols.append(col)\nprint(assetIds_cols)   \n    '

In [462]:
#Function to find columns with context actions, containing $ signs
def find_context_columns(row):
        if re.match('^\$[A-Za-z0-9]+\$\w{8}-\w{4}-\w{4}-\w{4}-\w{12}$', str(row)):
            return True   
        else:
            return False
"""
#find columns with rows containing $$
context_cols = []
for col in weapon_enchants_df.columns:    
        a = weapon_enchants_df[col].apply(find_context_columns).any()
        if a:            
            context_cols.append(col)
print(context_cols)   
    """

'\n#find columns with rows containing $$\ncontext_cols = []\nfor col in weapon_enchants_df.columns:    \n        a = weapon_enchants_df[col].apply(find_context_columns).any()\n        if a:            \n            context_cols.append(col)\nprint(context_cols)   \n    '

In [463]:
def find_item_type(row):
    """Function to return item type from the column "Data_$type"
    """
    tmp = str(row).split(',')
    data_type = tmp[1]    
    parts = re.split('(?<=[a-z])(?=[A-Z])', data_type)
    item_type = parts[-1]   
    return item_type

In [464]:
def create_text_columns(df, index_df, columns_in, column_out, format_text=False):
    """ Function to create text columns with names and descriptions got from 
        stringkeys in index file in the form of dict
        """
    # list of columns with string keys of items' names
    cols_names = columns_in
    #created column
    column_out = str(column_out )
    index_df = index_df # index_dict object got from parsing json file with text information
    index_dict = dict(zip(index_df['index'],index_df['text'])) #dict for mapping text information between two tables    
    
    #_____________________________Get items names and descriptions__________________________ 
    
    for col in cols_names:
        list_of_ids = df[col].unique() # list of stringkeys
        #filter out empty elements
        list_of_ids = list(filter(None,list_of_ids))
        for id in list_of_ids:
            try: #if stringkey is in index file return text
                df.loc[df[col]==id, column_out] = df[col].map(index_dict)
            except Exception as err:
                print(f'Unexpected {err=}, {type(err)=}',id) #if not return empty row
                df.loc[df[col]==id, column_out] = ''          
    
         # fill missed values with empty row       
    df[column_out] = df[column_out].fillna('')
    # format description to get rid of {}
    if format_text == True:
        df[column_out] = df[column_out].apply(format_description)

   
    return df

In [465]:
def find_property_cols(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_(?:Fact|Facts)(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False

"""
#find columns with rows containing property values
property_cols = []
for col in equip_items_df.columns:    
        a = equip_items_df[col].apply(find_property_cols).any()
        if a:            
            property_cols.append(col)
print(property_cols)   
    """    

'\n#find columns with rows containing property values\nproperty_cols = []\nfor col in equip_items_df.columns:    \n        a = equip_items_df[col].apply(find_property_cols).any()\n        if a:            \n            property_cols.append(col)\nprint(property_cols)   \n    '

In [466]:
def find_properties(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_[A-Za-z]+(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False
def select_columns(df, keywords):
    """function to select columns to keep from a dataframe and return 4 lists of
        stringkey columns, assetids columns, misc columns and combined list of columns
    Arguments: df - dataframe to choose columns from, 
                keywords - list of keywords or names of columns to select for misc list    
    """
    selected_cols = [] # list of needed columns
    assetIds_cols = [] #list of columns with asset ids
    stringkey_cols = [] # list of columns with string keyes
    misc_cols = [] # list of misc columns
    property_cols = [] # list of property columns
    intro_cols = []   # first columns: AssetId and Data_$type in any table

    def find_ids(col):
        if re.match('^[a-z0-9]{32}(?:$|[, A-Za-z])', str(col)):
            return True
        else:
            return False      

    #find columns with asset ids
    for col in df.columns:
        if col !='AssetId':
            a = df[col].apply(find_assetIds_cols).any()
            if a:            
                assetIds_cols.append(col)
        else:
            intro_cols.append(col)

    #find columns with string keys
    for col in df.columns:    
        a = df[col].apply(match_pattern).any()
        if a:            
            stringkey_cols.append(col)
    
    #find columns with rows containing property values

    for col in df.columns: 
        # if name of the column is according to the pattern   
        if  re.match('^Data_Components_[0-9]_(?:\$type$|m_[A-Za-z]+(?:$|_[0-9]$))',str(col)):
            a = df[col].apply(find_ids).any()
            if a:                        
        #and its values are assetIds keys  add to property list                 
                property_cols.append(col) 
      
    # find miscellanious columns
    if len(keywords)>0:
        keywords = keywords
        for keyword in keywords:
            for col in df.columns:
                if keyword in col:
                    misc_cols.append(col)
    #else:
        #misc_cols = [d for d in df.columns if d not in assetIds_cols if d not in stringkey_cols if d not in property_cols]
    #update assetIds_cols
    assetIds_cols = [d for d in assetIds_cols if d not in property_cols]
    #Must- have columns, 'AssetId' and 'Data_$type'
    intro_cols.append(df.columns[1]) #add Data_$type
    #create combined list            
    selected_cols = intro_cols + assetIds_cols+ property_cols + stringkey_cols + misc_cols + ['Filepath']
      
    return selected_cols, assetIds_cols, stringkey_cols, property_cols, misc_cols

In [467]:
def find_property_cols(row):
    #if  re.match('^Data_Components_[0-9]_\$type$',row) or re.match('^Data_Components_[0-9]_m_(?:Fact|Facts)(?:$|_[0-9]$)',row):   
    if re.match('^Data_Components_[0-9]_(?:\$type$|m_(?:Fact|Facts)(?:$|_[0-9]$))',str(row)):
        return True
    else:
        return False

"""
#find columns with rows containing property values
property_cols = []
for col in equip_items_df.columns:    
        a = equip_items_df[col].apply(find_property_cols).any()
        if a:            
            property_cols.append(col)
print(property_cols)   
    """    
       

'\n#find columns with rows containing property values\nproperty_cols = []\nfor col in equip_items_df.columns:    \n        a = equip_items_df[col].apply(find_property_cols).any()\n        if a:            \n            property_cols.append(col)\nprint(property_cols)   \n    '

In [468]:
def find_item_columns(df):
    #Function to find columns with names like Data_Items_0_m_Item
    item_cols = []
    for col in df.columns:
        try:
            if re.match('^Data_Items_[0-9]+_m_Item$',str(col)):
                item_cols.append(col)
        except Exception as err:
                    print(f'Unexpected {err=}, {type(err)=}',col)
            
    return item_cols

In [469]:
def drop_trash_columns (df):    
    df = df.loc[:,~df.where(df.astype(bool)).isna().all(axis = 0)]
    return df

In [470]:
def get_text_info(df, map_df, index_df, columns_in, keylist = None, return_temp = False, suffix = None, format_text=False, add_aggr_col = False):
    """ Function to create text columns with names and descriptions. Works on columns with asset ids
        and get filenames of those asset ids in asset_df and then look into files to extract strinkeys,
        max length of keylist is 4.
        Arguments:
        df - target df apply changes to;
        map_df - assets_df to get assets ids of files;
        index_df - dataframe of index file to get text info from;
        columns_in - list of columns apply changes over;
        keylist - keys of the received file get information from, if None search string keys to get ONLY names (not description keys);
        return_temp - if True return list of first columns with data, skipping further search and creation of columns;
                     if False, intermediate columns will be dropped.
        suffix - assign siffix to names of intermediate columns, if you return them, its better dont use 'text' for it,
                 because it is used inside function.
        format_text - if True apply format_description() function to the columns;
        add_aggr_col - create column with sum of values (or concatenation of strings) for provided columns;       
        """    
    # list of columns with string keys of items' names
    cols_names = columns_in
    #created column        
    # map_df - a dataframe for mapping    
    assets_dict = dict(zip(map_df.iloc[:,0],map_df.iloc[:,1])) #dict for mapping text information between two tables
    #reference to dataframe go from localization file    
    
    def find_parent(D):
        """find first key path to value containing stringkey"""    
        for k, v in D.items():        
            if isinstance(v,dict):
                parent = find_parent(v)
                if parent:
                    return [k] + parent
            elif match_pattern(v):                
                return [k]
    def find_stringkeys(row, keylist = None):    
        try:
            row = row.strip()
            with open(row) as asset_file:           
                file_contents = asset_file.read()
                data = json.loads(file_contents)
                try: 
                    if keylist is None:                   
                        keylist = find_parent(data)
                        if len(keylist)==3:
                            key = data[keylist[0]][keylist[1]][keylist[2]]
                            return key
                        else:
                            key = data[keylist[0]][keylist[1]][keylist[2]][keylist[3]]
                            return key
                    else:
                        if len(keylist)==1:
                            key = data[keylist[0]]
                            return key
                        if len(keylist)==2:
                            key = data[keylist[0]][keylist[1]]
                            return key
                        if len(keylist)==3:
                            key = data[keylist[0]][keylist[1]][keylist[2]]
                            return key
                        else:
                            key = data[keylist[0]][keylist[1]][keylist[2]][keylist[3]]
                            return key                  
                except:                         
                        return ''
                   
                                   
                
        except:
            return ''      
    
    #_____________________________Get items names and descriptions__________________________ 
    temp_list = []
    columns_out = []
    if suffix == None:
        suffix=='stringkey'
    for col in cols_names:                
                df[col] = df[col].str.strip()         
                filenames_series = df[col].map(assets_dict)               
                     
                df['{}_{}'.format(col,suffix)]  = filenames_series.apply(find_stringkeys, keylist = keylist)
                

                temp_list.append('{}_{}'.format(col,suffix))
                text_column_name = '{}_text'.format(col)
                columns_out.append(text_column_name)
    if return_temp == True:
          if format_text == True:            
            df[temp_list] = df[temp_list].apply(format_description,format_text = format_text)
            return df
          else:
              return df
    else:
         
        for i in range(len(columns_out)):
            col_in = [temp_list[i]]
            df = create_text_columns(df, index_df, col_in, columns_out[i], format_text = format_text)
        #drop created stringkey cols
        df = df.drop(columns = temp_list, axis = 1)
        if add_aggr_col == True:
            df['All_Items'] =df[columns_out].agg(sum)  
        return df, columns_out

In [471]:
def get_talent_class(filepath):
    """Function to extract class name from filepath"""
    def camel_case_split(str):
        """Function useful for splitting camelcase words""" 
        return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)
    #get the filename
    head, tail = os.path.split(filepath)
    parts = camel_case_split(tail)
    return parts[0]
"""
#Example:
#head, tail = os.path.split(filepath)
row = 'e:\Steam\RogueTrader\WorkFolder\Blueprints\Classes\Assassin\AssassinTalents\Assassin_FlawlessLunge_Talent.jbp'
head, tail = os.path.split(row)
print(tail)
camel_case_split(tail)
"""

"\n#Example:\n#head, tail = os.path.split(filepath)\nrow = 'e:\\Steam\\RogueTrader\\WorkFolder\\Blueprints\\Classes\\Assassin\\AssassinTalents\\Assassin_FlawlessLunge_Talent.jbp'\nhead, tail = os.path.split(row)\nprint(tail)\ncamel_case_split(tail)\n"

### Folders

In [472]:
#folder with the gameobjects' parameters
work_folder = r'e:\Steam\RogueTrader\WorkFolder\Blueprints'
# file with the english localization
index_file = r'e:\Steam\RogueTrader\Localization\enGB.json'
#file with the russian localization
index_file_ru = r'e:\Steam\RogueTrader\Localization\ruRU.json'
save_folder = r'e:\Steam\RogueTrader\SaveFolder'
load_folder = r'e:\Steam\RogueTrader\LoadFolder'
folder_dialogs = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs'


### Support tables

In [473]:
#loading support dataframes
assets_df = pd.read_csv(load_folder + r'\assets_df.csv')
index_df = pd.read_csv(load_folder + r'\index_df.csv')
index_df_ru = pd.read_csv(load_folder + r'\index_df_ru.csv')
# english version equipment table
equip_items_df = pd.read_csv(load_folder + r'\equip_items_df.csv')
#english version weapon table
weapons_table = pd.read_csv(load_folder + r'\weapons.csv')
#english version armor table
armors_df = pd.read_csv(load_folder + r'\armors_df.csv')
#areas data
areas_data = pd.read_csv(load_folder + r'\areas_data.csv')

## Work Part

### Creating table

In [474]:
# list of files in work_folder
list_of_files = []
for root, dirs, files in os.walk(folder_dialogs):           
        for file in files:
            if file.endswith(".jbp"):            
                list_of_files.append(os.path.join(root, file))
print(len(list_of_files))

36115


In [475]:
key_name = 'm_ItemToGive' #target key value
def get_all_keys(d):
    """
    The function to get keys from nested dict containing specific value    
    """
    for key, value in d.items():
        if key==key_name:                          
            yield d[key]
        else:        
            if isinstance(value, dict):                     
                     yield from get_all_keys(value)
            elif isinstance(value, list):
                    for ele in value:
                        if isinstance(ele, dict):                            
                                yield from get_all_keys(ele)           
dialog_items = {} 
for file in list_of_files:
        with open(file, encoding='latin-1') as user_file:
            file_contents = user_file.read()
            data = json.loads(file_contents)            
            file_info = {}
            file_info['Filepath'] = file                                 
            try:
                values_list = list(get_all_keys(data))
                if values_list:
                    list_of_items = []
                    for i in values_list:
                        item_id = clear_rows(i)
                        list_of_items.append(item_id)
                    
                    file_info['ParentAsset'] = data.get("Data").get('ParentAsset')                   
                    file_info['Text_stringkey'] = data.get('Data').get('Text').get('m_Key')                   
                    file_info['ItemGiverId'] = clear_rows(data.get('Data').get('Speaker').get('m_Blueprint'))
                else:
                    list_of_items = np.NAN                                        
                file_info['Items'] = list_of_items                                          
                dialog_items[data['AssetId']] = file_info
            except Exception as err:
                    print(f'Unexpected {err=}, {type(err)=}', file)
# create dataframe with files containing items to give                    
dialog_items_df = pd.DataFrame.from_dict(dialog_items, orient = 'index')
dialog_items_df = dialog_items_df[dialog_items_df['Items'].notna()].reset_index().rename(columns = {'index':'AssetId'})
# create column with lenghth of lists in the column 'Items'
dialog_items_df['Number_Items'] = dialog_items_df['Items'].map(len)

Unexpected err=AttributeError("'NoneType' object has no attribute 'get'"), type(err)=<class 'AttributeError'> e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch1\RaycadMinoris\ArcheotechReactor\Aurora\Answer_0002.jbp
Unexpected err=AttributeError("'NoneType' object has no attribute 'get'"), type(err)=<class 'AttributeError'> e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch2_VS\Dargonus\Ceremony_Coronation\WinterscaleChorda\Answer_0005.jbp
Unexpected err=AttributeError("'NoneType' object has no attribute 'get'"), type(err)=<class 'AttributeError'> e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch2_VS\Dargonus\RogueTraderPalace\DargonusFirstArrival\DargonusFirstArrival_dialogue.jbp
Unexpected err=AttributeError("'NoneType' object has no attribute 'get'"), type(err)=<class 'AttributeError'> e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch2_VS\DaringHeirGuttedShip\NotDoneWithYou\Answer_0005.jbp
Unexpected err=AttributeError("'NoneType' object has no

key_name = 'm_ItemToGive' #target key value
def get_all_keys(d):
    """
    The function to get keys from nested dict containing specific value    
    """
    for key, value in d.items():
        if key==key_name:                          
            yield d[key]
        else:        
            if isinstance(value, dict):                     
                     yield from get_all_keys(value)
            elif isinstance(value, list):
                    for ele in value:
                        if isinstance(ele, dict):                            
                                yield from get_all_keys(ele)           
dialog_items = {} 
for file in list_of_files:
        with open(file, encoding='latin-1') as user_file:
            file_contents = user_file.read()
            data = json.loads(file_contents)            
            file_info = {}
            file_info['Filepath'] = file                                 
            try:
                values_list = list(get_all_keys(data))
                if values_list:
                    list_of_items = []
                    for i in values_list:
                        item_id = clear_rows(i)
                        list_of_items.append(item_id)
                    if data["Data"]["ParentAsset"] is not None:
                        file_info['ParentAsset'] = data["Data"]["ParentAsset"]
                    else:
                        file_info['ParentAsset'] = np.NaN
                    if data['Data']['Text']['m_Key'] is not None:
                        file_info['Text_stringkey'] = data['Data']['Text']['m_Key']
                    else:
                        file_info['Text_stringkey'] = np.NaN
                    if data['Data']['Speaker']['m_Blueprint'] is not None:
                        file_info['ItemGiverId'] = clear_rows(data['Data']['Speaker']['m_Blueprint'])
                    else:
                        file_info['ItemGiverId'] = np.NaN
                else:
                    list_of_items = np.NAN                                        
                file_info['Items'] = list_of_items                                          
                dialog_items[data['AssetId']] = file_info
            except Exception as err:
                    print(f'Unexpected {err=}, {type(err)=}', file)
# create dataframe with files containing items to give                    
dialog_items_df = pd.DataFrame.from_dict(dialog_items, orient = 'index')
dialog_items_df = dialog_items_df[dialog_items_df['Items'].notna()].reset_index().rename(columns = {'index':'AssetId'})
# create column with lenghth of lists in the column 'Items'
dialog_items_df['Number_Items'] = dialog_items_df['Items'].map(len)
 

In [476]:
# get max value of elements in lists
dialog_items_df['Number_Items'].max()

5

Create separate columns for each item in lists:

In [477]:
dialog_items_df[['Item_0',
              'Item_1','Item_2',
              'Item_3','Item_4']] = pd.DataFrame(dialog_items_df['Items'].tolist(), index = dialog_items_df.index)

In [478]:
dialog_items_df.query('Number_Items==5')

AssetId  \
41  8ae87b121dd44a298b84eb8384761039   

                                                                                                   Filepath  \
41  e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch2_VS\VoidEvents\AdMechCapsized_BE\Cue_16.jbp   

                                                                                                                                                                         Items  \
41  [f7c3eb54e48d4f1fbbabef9a7b863f56, 7ddc00066e424196b36e11fb8e434d04, b0a659a6de934dc4970694eabe9b7a2e, b6339c410d984c81b7c487081aed183e, 86bc9833350c406f85531f4c4d2e65c8]   

                         ParentAsset                        Text_stringkey  \
41  cbd95e6d570146abb9263b9d978774e5  1bcfc828-43b3-43ed-97d0-fedd11aed5de   

   ItemGiverId  Number_Items                            Item_0  \
41        None             5  f7c3eb54e48d4f1fbbabef9a7b863f56   

                              Item_1                            Item_2  \
41  7ddc00066e424196b36e11fb8e434d04  b0a659a6de934dc4970694eabe9b7a2e   

                              Item_3                            Item_4  
41  b6339c410d984c81b7c487081aed183e  86bc9833350c406f85531f4c4d2e65c8

In [479]:
item_columns = ['Item_0','Item_1','Item_2', 'Item_3','Item_4']
dialog_items_df, columns = get_text_info(df = dialog_items_df,
                            map_df = assets_df[['AssetId','File']],
                            index_df = index_df,
                            columns_in = item_columns)


In [480]:
dialog_items_df.head()

AssetId  \
0  c288ebc4d447499ab50e5caeb7424a1e   
1  3dd010e713b84ce0b25b77ebaf74fa58   
2  5deaa26a4ce543f48aee49b5e2f9e35f   
3  e4572b5a9c8842aabebbb8259fd3817e   
4  5ce299b3395845279ee8fbb4af43a874   

                                                                                               Filepath  \
0        e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch0_Prologue\AbelardRecruit\Cue_6.jbp   
1      e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch0_Prologue\TheodorasFate\Cue_0030.jbp   
2   e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch1\BridgeAndCabinet\Briefing\Cue_0007.jbp   
3  e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch1\NavisNobiliteStation\Felec\Cue_0002.jbp   
4  e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch1\NavisNobiliteStation\Felec\Cue_0047.jbp   

                                Items                       ParentAsset  \
0  [56e3576b84384495b9015d5ec88217ac]  98d3d3e4656a411981204d81617af22d   
1  [b9f39321b65d4f16af45974a742d438e]  7b5f7c004fb44656ba4f34130f84284d   
2  [ef261dfb9dfc44829841cd89b533dc66]  80387af65af3426eac205956f20078af   
3  [b8e43fdbe5ab4b09a8754af2816db861]  f2f195adf10747ffa2d465aa86dd24c6   
4  [b8e43fdbe5ab4b09a8754af2816db861]  8fcd72613c564180a8f893615b7af6a8   

                         Text_stringkey ItemGiverId  Number_Items  \
0  1483e02a-c1a1-4998-936d-b44242b097c5        None             1   
1  9f024ca3-98c7-4612-bf50-91925611954c        None             1   
2                                              None             1   
3  26736dcb-8b3b-4830-8820-a6c4cc24db7e        None             1   
4  851f0530-1180-4f15-aab5-d73cd9cd5ae5        None             1   

                             Item_0 Item_1 Item_2 Item_3 Item_4  \
0  56e3576b84384495b9015d5ec88217ac   None   None   None   None   
1  b9f39321b65d4f16af45974a742d438e   None   None   None   None   
2  ef261dfb9dfc44829841cd89b533dc66   None   None   None   None   
3  b8e43fdbe5ab4b09a8754af2816db861   None   None   None   None   
4  b8e43fdbe5ab4b09a8754af2816db861   None   None   None   None   

                      Item_0_text Item_1_text Item_2_text Item_3_text  \
0                         Medikit                                       
1  Handwritten Letter With a Seal                                       
2                     Blade Shard                                       
3                   Glowing Stone                                       
4                   Glowing Stone                                       

  Item_4_text  
0              
1              
2              
3              
4

Meld dataframe into long format:

In [481]:
#eng version
dialog_items_df = pd.melt(dialog_items_df,id_vars=['AssetId', 'ParentAsset', 
                            'Text_stringkey', 'ItemGiverId', 'Filepath'],
                            value_vars=item_columns,
                            var_name = 'Item_Number',
                            value_name = 'Item_AssetId')

Create item's type column

In [482]:
weapons = weapons_table['AssetId'].unique().tolist()
armors = armors_df['AssetId'].unique().tolist()
equipment = equip_items_df['AssetId'].unique().tolist()
dialog_items_df.loc[dialog_items_df['Item_AssetId'].isin(weapons),'Item_Type'] = 'Weapon'
dialog_items_df.loc[dialog_items_df['Item_AssetId'].isin(armors),'Item_Type'] = 'Armor'
dialog_items_df.loc[dialog_items_df['Item_AssetId'].isin(equipment),'Item_Type'] = 'Equipment'

Create separate dataframes for each type of item:

In [483]:
dialog_weapons = dialog_items_df.query('Item_Type=="Weapon"')
dialog_armors = dialog_items_df.query('Item_Type=="Armor"')
dialog_equipment = dialog_items_df.query('Item_Type=="Equipment"')
dialog_items_others = dialog_items_df.query('Item_Type.isna()')

Merge created dataframes with imported tables of equipment to get item_names:

In [484]:
dialog_weapons = dialog_weapons.merge(weapons_table[['AssetId','Item_Name']], 
                                                    how = 'left',
                                                    left_on = 'Item_AssetId',
                                                    right_on = 'AssetId',
                                                    suffixes = (None,'_y')).drop('AssetId_y', axis = 1)
dialog_armors = dialog_armors.merge(armors_df[['AssetId','Item_Name']], 
                                                    how = 'left',
                                                    left_on = 'Item_AssetId',
                                                    right_on = 'AssetId',
                                                    suffixes = (None,'_y')).drop('AssetId_y', axis = 1)
dialog_equipment = dialog_equipment.merge(equip_items_df[['AssetId','Item_Name']], 
                                                    how = 'left',
                                                    left_on = 'Item_AssetId',
                                                    right_on = 'AssetId',
                                                    suffixes = (None,'_y')).drop('AssetId_y', axis = 1)
                                                                                                        

Create one dataframe with new data received:

In [485]:
dialog_items_table = pd.DataFrame()
dfs = [dialog_weapons, dialog_armors, dialog_equipment]
for df in dfs:
    if df is not None: # dont join empty dataframes
        dialog_items_table = pd.concat([dialog_items_table, df])
dialog_items_table = dialog_items_table.reset_index(drop = True)
display(dialog_items_table['Item_Type'].unique())

array(['Weapon', 'Equipment'], dtype=object)

In [486]:
map_dict = dict(zip(index_df.iloc[:,0],index_df.iloc[:,1]))
dialog_items_table['Text'] = dialog_items_table['Text_stringkey'].map(map_dict)
dialog_items_table['Text'] = dialog_items_table['Text'].astype('str')
dialog_items_table['Text'] = dialog_items_table['Text'].apply(format_description) 


In [487]:
#list of keys containing names
keylist = ['Data','LocalizedName','stringkey']
dialog_items_table, created_columns = get_text_info(df = dialog_items_table, 
                                  map_df = assets_df,
                                  index_df= index_df,
                                  columns_in= ['ItemGiverId'],
                                  keylist= keylist, format_text=True)
dialog_items_table = dialog_items_table.rename(columns = {'ItemGiverId_text':'ItemGiver_En'})
# create column for russian names                                 
dialog_items_table, created_columns = get_text_info(df = dialog_items_table, 
                                  map_df = assets_df,
                                  index_df= index_df_ru,
                                  columns_in= ['ItemGiverId'],
                                  keylist= keylist, suffix = '_textRu',
                                   format_text=True)
dialog_items_table = dialog_items_table.rename(columns = {'ItemGiverId_text':'ItemGiver_Ru'})

Create column with item location

In [488]:
row = r'e:\\Steam\\RogueTrader\\WorkFolder\\Blueprints\\World\\Dialogs\\Ch3\\Chasm\\MeatDump_WakeUp\\Cue_2.jbp'
row1 = r'e:\\Steam\\RogueTrader\\WorkFolder\\Blueprints\\World\\Dialogs\\Companions\\CompanionQuests\\Ulfar\\Ulfar_Q1\\KTC_TorbaldFinalConversation\\Cue_0039.jbp'
def calc_chapter(filepath):
    try:
        chapter = re.search('(Ch\d{1})',filepath).group()
    except:
        chapter = ""
    return chapter
def calc_place(filepath):
    try:
        if 'Ceremony' in filepath:
            place = 'Dargonus, Ceremony of coronation'            
        elif 'Bridge_ChapterFinale' in filepath:
            place = 'Voidship Bridge'
        elif 'AdMechCapsized' in filepath:
            place = 'The system "Agathus Archides", drifting ship'
        else:
            place = re.search('(?:Dargonus|Foulstone|Chasm|Eufrates|Quetza_al_Khet|Footfall)',filepath).group()            
    except:
        place = ""
    return place
calc_place(row)
dialog_items_table['Loc_Chapter'] = dialog_items_table['Filepath'].apply(calc_chapter)
dialog_items_table['Loc_Place'] = dialog_items_table['Filepath'].apply(calc_place)

In [489]:
#dialog_items_table.query('Loc_Place =="" | Loc_Chapter==""')
#print(dialog_items_table.shape[0])

### Create Location column

In [490]:
# dict for mapping values
companion_items = {
    "a223f91296d24d65a3390651c247ed83":"Chapter 2, gift from Jae after helping her on Footfall.",
    "4ec59f09523148f98cb05d63f63e5d4f":"Chapter 2, gift from Jae after helping her on Footfall.",
    "67db0f23724442fbae91a21191114c54":"Gift from Thorbald to Ulfar after receiving the horm of the Helbrute.",
    "4d1a06582e3147d09228433ac3882a63":"Chapter 2, one of artefacts got from missed Aeldari ships during Yrliet's quest.",
    "a7b71b397d8f437cad9442b53cc6fdad":"Sacrifice Argenta to the Edge of Daybreak.",
    "b711dacb0c2f4af58b7ad0292d6e2f6b":"Chapter 2, reward from jae after completing her 2nd quest.",
    "36e46c77f00f4a5b98af33842cafebf1":"Sacrifice Argenta to the Edge of Daybreak.",
    "862e2050234547ed9b53e0ee75e25875":"Chapter 4, the reward from Falco if he receives the Mercatula Tabula Oficiale from Jae's body.",
    "55f78d15bf4348688620d74d9623e5ac":"Chapter 4, finding the culprit in Abelard's quest.",
    "c00a36ee30bc4552be7f437856c48b0c":"Sacrifice Argenta to the Edge of Daybreak.",
    "19c9755559574a7ea34adb408f8c5c06":"Chapter 2, let the navigator Elina go after confronting her in the palace.",
    "c36e22c76e7c4a3aabdc397ad319ba54":"Chapter 3, gift from Cassia, if player has romance with her.", 
    "549d7fa415204658859486c29b2d6dbe":"Sacrifice Argenta to the Edge of Daybreak.",
    "ddbef450b07a4965ab21939dc9d69954":"Dargonus, Clementia, if player reached rank III iconoclast.",
    "9a9d4d64022e47189676dfba000c3ef8":"Dargonus, Clementia, if player reached rank III heretic.",
    "5eabde7889e845c39459eaf845df903b":"Dargonus, Clementia, if player reached rank III dogmatic.",
    "c704ead32a2241218dc911ee60f520c7":"Chapter 4, Quetza_al_Khet, Idira's quest.",
    "082334e491e34c65b464c90e317d4e7f":"Ch2, The system Agathus Archides, drifting ship, the 2nd team succefully returns to ship.",
    "05159922d36a4018bdcab94a4a3f979c":
    "Ch2, The system Agathus Archides, drifting ship, the 2nd team succefully returns to ship.",
    "3abd217a363248028aacf4e220f49fdd":
    "Ch2, The system Agathus Archides, drifting ship, the Mattina's group succefully returns to ship.",
    "86bc9833350c406f85531f4c4d2e65c8":
    "Ch2, The system Agathus Archides, drifting ship, the Captain Vesti's group succefully returns to ship.",

}
companion_items_ru = {
    "a223f91296d24d65a3390651c247ed83":"Глава 2, подарок Джай в ответ на помощь на Футфоле.",
    "4ec59f09523148f98cb05d63f63e5d4f":"Глава 2, подарок Джай в ответ на помощь на Футфоле.",
    "67db0f23724442fbae91a21191114c54":"Подарок Торбальда Ульфару после убийства хелбрута.",
    "4d1a06582e3147d09228433ac3882a63":"Глава 2, один из артефактов, полученных в ходе выполнения квеста Ирлиет.",
    "a7b71b397d8f437cad9442b53cc6fdad":"Принести Арженту в жертву для Предела Рассвета.",
    "b711dacb0c2f4af58b7ad0292d6e2f6b":"Глава 2, подарок Джай после выполнения её второго квеста.",
    "36e46c77f00f4a5b98af33842cafebf1":"Принести Арженту в жертву для Предела Рассвета.",
    "862e2050234547ed9b53e0ee75e25875":"Глава 4, подарок Фалько, если он получит патент с тела Джай.",
    "55f78d15bf4348688620d74d9623e5ac":"Глава 4, награда за завершение квеста Абелярда на Даргонусе.",
    "c00a36ee30bc4552be7f437856c48b0c":"Принести Арженту в жертву для Предела Рассвета.",
    "19c9755559574a7ea34adb408f8c5c06":"Глава 2, подарок навигатора Элины, если игрок позволит ей уйти из дворца.",
    "c36e22c76e7c4a3aabdc397ad319ba54":"Глава 3, подарок Кассии, если у игрока с ней роман.", 
    "549d7fa415204658859486c29b2d6dbe":"Принести Арженту в жертву для Предела Рассвета.",
    "ddbef450b07a4965ab21939dc9d69954":"Даргонус, Клеменция, если игрок достиг ранг 3 схизматика.",
    "9a9d4d64022e47189676dfba000c3ef8":"Даргонус, Клеменция, если игрок достиг ранг 3 еретика.",
    "5eabde7889e845c39459eaf845df903b":"Даргонус, Клеменция, если игрок достиг ранг 3 догматика.",
    "c704ead32a2241218dc911ee60f520c7":"Глава 4, Кетца-Темер, квест Идиры.",
    "082334e491e34c65b464c90e317d4e7f":
    "Глава 2, Система Агатус Архидес, событие на покинутом корабле, вторая группа успешно вернулась.",
    "05159922d36a4018bdcab94a4a3f979c":
    "Глава 2, Система Агатус Архидес, событие на покинутом корабле, вторая группа успешно вернулась.",
    "3abd217a363248028aacf4e220f49fdd":
    "Глава 2, Система Агатус Архидес, событие на покинутом корабле, группа Маттины успешно вернулась.",
    "86bc9833350c406f85531f4c4d2e65c8":
    "Глава 2, Система Агатус Архидес, событие на покинутом корабле, группа капитана Вести успешно вернулась.",         
}


In [491]:
#column for companion items
dialog_items_table['Item_Location'] = dialog_items_table['Item_AssetId'].map(companion_items)
# compaund column from columns got from filepath string parsing
dialog_items_table['Item_Loc_Temp'] =( dialog_items_table['Loc_Chapter']+', '+ dialog_items_table['Loc_Place'] 
                                    +', '+dialog_items_table['ItemGiver_En']
                                        )
dialog_items_table['Item_Location'] = dialog_items_table['Item_Location'].fillna(dialog_items_table['Item_Loc_Temp'])
dialog_items_table['Item_Location'] = dialog_items_table['Item_Location'].str.rstrip().str.rstrip(',')

### Create location columns for russian version

Making dicts for mapping values:

In [492]:
print(dialog_items_table['Loc_Chapter'].unique())
print(dialog_items_table['Loc_Place'].unique())

['Ch2' 'Ch3' 'Ch4' '']
['Dargonus, Ceremony of coronation' 'Foulstone'
 'The system "Agathus Archides", drifting ship' 'Chasm' 'Voidship Bridge'
 'Eufrates' 'Footfall' 'Quetza_al_Khet' '' 'Dargonus']


In [493]:
#places
chapters_dict = {'Ch2':'Глава 2',
                 'Prologue':'Пролог',
                 'Ch3':'Глава 3',
                 'Ch4':'Глава 4',
                 'Ch5':'Глава 5',   
                }
places_dict = {'Dargonus, Ceremony of coronation':'Даргонус, церемония коронации',
                 'Foulstone':'Фолстоун',
                 'The system "Agathus Archides", drifting ship':'Система Агатус Архидес, событие на покинутом корабле',
                 'Chasm':'Прорва',
                 'Voidship Bridge':'Мостик корабля', 
                 'Eufrates':'Ефрат II', 
                 "Footfall":"Футфол",
                 "Quetza_al_Khet":"Кетца-Темер",
                 "Dargonus":"Даргонус", 
                }
dialog_items_table['Loc_Place_ru'] = dialog_items_table['Loc_Place'].map(places_dict)
dialog_items_table['Loc_Chapter_ru'] = dialog_items_table['Loc_Chapter'].map(chapters_dict)
#column for companion items
dialog_items_table['Item_Location_ru'] = dialog_items_table['Item_AssetId'].map(companion_items_ru)

dialog_items_table['Item_Loc_Temp_ru'] =( dialog_items_table['Loc_Chapter_ru']+', '+ dialog_items_table['Loc_Place_ru'] 
                                    +', '+dialog_items_table['ItemGiver_Ru']
                                        )
dialog_items_table['Item_Location_ru'] = dialog_items_table['Item_Location_ru'].fillna(dialog_items_table['Item_Loc_Temp_ru'])
dialog_items_table['Item_Location_ru'] = dialog_items_table['Item_Location_ru'].str.rstrip().str.rstrip(',')

In [494]:
display(dialog_items_table['Item_Location_ru'].unique())

array(['Глава 2, Даргонус, церемония коронации, Губернатор Урбенд Драйвстем',
       'Глава 2, Даргонус, церемония коронации', 'Глава 2, Фолстоун',
       'Глава 2, Система Агатус Архидес, событие на покинутом корабле, вторая группа успешно вернулась.',
       'Глава 3, Прорва', 'Глава 3, Прорва, Неизвестный',
       'Глава 4, Мостик корабля, Каллигос Винтерскейл',
       'Глава 4, Мостик корабля, Инцендия Чорда', 'Глава 4, Ефрат II',
       'Глава 4, Футфол, Оптикон-22',
       'Глава 4, Футфол, Наместник Владайм Токара',
       'Глава 4, Футфол, Октавиана', 'Глава 4, Футфол, Дзета-42//33-Мю',
       'Глава 4, Кетца-Темер, Вортен Серый',
       'Глава 2, подарок Джай в ответ на помощь на Футфоле.',
       'Подарок Торбальда Ульфару после убийства хелбрута.',
       'Глава 2, один из артефактов, полученных в ходе выполнения квеста Ирлиет.',
       'Глава 4, Кетца-Темер',
       'Принести Арженту в жертву для Предела Рассвета.',
       'Глава 2, Даргонус, церемония коронации, Вицент Гап

In [495]:
display(dialog_items_table[dialog_items_table['Filepath'].str.contains('AdMechCapsized_BE')])

AssetId                       ParentAsset  \
3   ffd056771f90468eab08509fb20eaecb  a11bd69a9426443eaee1aafba4da6a63   
29  ffd056771f90468eab08509fb20eaecb  a11bd69a9426443eaee1aafba4da6a63   
71  f914907befbd4784a15dd916bab6f158  d8bff69c5ec1412cbf30a2a5dc64432c   
72  8ae87b121dd44a298b84eb8384761039  cbd95e6d570146abb9263b9d978774e5   

                          Text_stringkey ItemGiverId  \
3   346168ce-9e68-40ef-9b9b-ef49bafe433b        None   
29  346168ce-9e68-40ef-9b9b-ef49bafe433b        None   
71  049baa67-453e-4db7-86d7-e1685f60868f        None   
72  1bcfc828-43b3-43ed-97d0-fedd11aed5de        None   

                                                                                                     Filepath  \
3   e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch2_VS\VoidEvents\AdMechCapsized_BE\Cue_0022.jbp   
29  e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch2_VS\VoidEvents\AdMechCapsized_BE\Cue_0022.jbp   
71  e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch2_VS\VoidEvents\AdMechCapsized_BE\Cue_0027.jbp   
72    e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch2_VS\VoidEvents\AdMechCapsized_BE\Cue_16.jbp   

   Item_Number                      Item_AssetId  Item_Type  \
3       Item_0  082334e491e34c65b464c90e317d4e7f     Weapon   
29      Item_3  05159922d36a4018bdcab94a4a3f979c     Weapon   
71      Item_2  3abd217a363248028aacf4e220f49fdd  Equipment   
72      Item_4  86bc9833350c406f85531f4c4d2e65c8  Equipment   

                    Item_Name  \
3          Hot Shot Laspistol   
29                    Longlas   
71  Headhunter Light Carapace   
72             Tactical Cloak   

                                                                                                                                                                                                                                                                                                           Text  \
3                                                                                                                                                    The second unit had succeeded in moving through the danger zone and had returned to the shuttle with a valuable payload taken from the dead ship's stores.   
29                                                                                                                                                   The second unit had succeeded in moving through the danger zone and had returned to the shuttle with a valuable payload taken from the dead ship's stores.   
71                                 Fortune smiled on the unit this time. Grey Mattina, having trusted her instinct and experience, chose to return to the shuttle via an alternate route. In one of the dark branching passageways, the team discovered a modest but untouched trove of cargo from Kiava Gamma.   
72  The Lord Captain's assessment was proved entirely justified — with difficulty but no adverse incidents, the unit crossed the dangerous section. Their visit to the storage depot was just as fortuitous — it was full of cargo. Captain Vesti's satisfied smile made it clear he believe this result was...   

   ItemGiver_En ItemGiver_Ru Loc_Chapter  \
3                                    Ch2   
29                                   Ch2   
71                                   Ch2   
72                                   Ch2   

                                       Loc_Place  \
3   The system "Agathus Archides", drifting ship   
29  The system "Agathus Archides", drifting ship   
71  The system "Agathus Archides", drifting ship   
72  The system "Agathus Archides", drifting ship   

                                                                                             Item_Location  \
3                Ch2, The system Agathus Archides, drifting ship, the 2nd team succefully returns to ship.   
29               Ch2, The system Agathus Archides, drifting ship, the 2nd team succefully r

### Exploring data on the example of ```Eye of Hecaton```

The item's assest id: ```7e7046a277ee4fc8af4717ff81fa8b1e```    
['Data]['Text']['m_Key'] - the phrase which provides the item.     
['Data']['Speaker']['m_Blueprint'] - the speaker of the phrase.    
['Data']['Continue']['Cues'] - next phrase after the player clicks 'Continue'.    
['Data']['Answers'] - list of possible answers to the phrase.    
['Data']['ParentAsset'] - id of the file that contains player character's previous phrase.   
['Data']['ParentAsset'] ->['Data]['Text']['m_Key'] - previous phrase text. 
['Data']['SoulMarkRequirement']['Direction']   
['Data']['SoulMarkRequirement']['Value'] - requirements of convictions.  

If previous step is a answer list, then ParentAsset key leads to the file of  
 ```"$type": "52ca6275d0a87bf428811a8c24d89de4, BlueprintAnswersList",``` with 
['Data]['Answers'] - list of possible answers on the previous step.  

The root i file with datatype ```"$type": "c8ff73feae580b142a9f43e0c61d7f32, BlueprintDialog"```



In [496]:
dialog_items_table[dialog_items_table['Item_Location']==', , ']

Empty DataFrame
Columns: [AssetId, ParentAsset, Text_stringkey, ItemGiverId, Filepath, Item_Number, Item_AssetId, Item_Type, Item_Name, Text, ItemGiver_En, ItemGiver_Ru, Loc_Chapter, Loc_Place, Item_Location, Item_Loc_Temp, Loc_Place_ru, Loc_Chapter_ru, Item_Location_ru, Item_Loc_Temp_ru]
Index: []

In [497]:
dialog_items_table['Item_Location'].unique()

array(['Ch2, Dargonus, Ceremony of coronation, Governor Urbend Drivestem',
       'Ch2, Dargonus, Ceremony of coronation', 'Ch2, Foulstone',
       'Ch2, The system Agathus Archides, drifting ship, the 2nd team succefully returns to ship.',
       'Ch3, Chasm', 'Ch3, Chasm, Stranger',
       'Ch4, Voidship Bridge, Calligos Winterscale',
       'Ch4, Voidship Bridge, Incendia Chorda', 'Ch4, Eufrates',
       'Ch4, Footfall, Opticon-22',
       'Ch4, Footfall, Vladaym Tocara the Liege',
       'Ch4, Footfall, Octaviana', 'Ch4, Footfall, Zeta-42//33-Mu',
       'Ch4, Quetza_al_Khet, Worten the Grey',
       'Chapter 2, gift from Jae after helping her on Footfall.',
       'Gift from Thorbald to Ulfar after receiving the horm of the Helbrute.',
       "Chapter 2, one of artefacts got from missed Aeldari ships during Yrliet's quest.",
       'Ch4, Quetza_al_Khet',
       'Sacrifice Argenta to the Edge of Daybreak.',
       'Ch2, Dargonus, Ceremony of coronation, Vicent Gaprak',
       'Ch2,

_______________

In [498]:
# etude f1b11c55ca3c4344add79f0933ce451c
assets_df.query('AssetId=="114c13cdc77241f2a6c558e8e4e5d758"')['File']

107968    e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Etudes\RogueTraderEtude\SoulMark\SoulMark\RTPalace_Hope.jbp
Name: File, dtype: object

In [499]:
# etude f1b11c55ca3c4344add79f0933ce451c
assets_df.query('AssetId=="a95f576d300a47808b6152ba541047e9"')['File']

99791    e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\SoulMarksEvents\DargonusReactivity\DargonusReactivity_dialogue\Cue_0006.jbp
Name: File, dtype: object

In [500]:
assets_df.query('AssetId=="c704ead32a2241218dc911ee60f520c7"')['File']

5310    e:\Steam\RogueTrader\WorkFolder\Blueprints\Equipment\Headgear\PsyJammer_Item.jbp
Name: File, dtype: object

In [501]:
index_df_ru.query('index=="1bcfc828-43b3-43ed-97d0-fedd11aed5de"')['text']

5680    Расчет Лорд-капитана полностью оправдался — отряд с усилиями, но без происшествий преодолел опасный участок. Столь же обнадеживающим оказался и визит в хранилище — оно было заполнено грузом. По довольной улыбке капитана Вести было ясно, что он уверен — это результат его молитвы.
Name: text, dtype: object

_________

### Saving

In [503]:
dialog_items_table.to_csv(save_folder + r'\dialog_items.csv', index = False)

______

In [502]:
filename = r'e:\Steam\RogueTrader\WorkFolder\Blueprints\World\Dialogs\Ch2_VS\Dargonus\Ceremony_Coronation\WinterscaleChorda\Answer_0005.jbp'
key_name = 'm_ItemToGive' #target key value
key_value = "!bp_d043a2289a1f45f784a415057c725cb5" # target key name
with open(filename) as user_file:
            file_contents = user_file.read()
            data = json.loads(file_contents)  

def findkeys(data, kv,key):
            """function to find parent dictionary containing a specific key-value pair
            kv - key value, 
            key - key name.
            """
            if isinstance(data, list):
                for i in data:
                    for x in findkeys(i, kv,key):
                        yield x
            elif isinstance(data, dict):
                if (key,kv) in data.items(): # check if key-value pair exists
                    yield data
                for j in data.values():
                    for x in findkeys(j, kv,key):
                        yield x
# output result
list(findkeys(data,key_value, key_name))

[{'$type': '1f1c883ef085c5c4b877eeb4c5da318c, AddItemToPlayer',
  'name': '$AddItemToPlayer$c1bedc77-c1db-4f0d-89c4-b90d5e0ded5c',
  'm_ItemToGive': '!bp_d043a2289a1f45f784a415057c725cb5',
  'Silent': False,
  'Quantity': 1,
  'Identify': False,
  'Equip': False,
  'EquipOn': None,
  'PreferredWeaponSet': 0,
  'ErrorIfDidNotEquip': True}]

#finding the file with a particular string searching through folders:
from pathlib import Path

path = Path(r'e:\Steam\RogueTrader\WorkFolder\Blueprints')

search_string = '39c107a0b6684325838b41f2fe0fa40c'

for o in path.rglob('*.jbp'):
    if o.is_file():
        text = o.read_text(encoding='latin-1')
        if search_string in text:
            print(o)
            print(text)